In [1]:
#Install requirements
!unzip archive2


Archive:  archive2.zip
   creating: archive2/BBC News Summary/
   creating: archive2/BBC News Summary/BBC News Summary/
   creating: archive2/BBC News Summary/BBC News Summary/News Articles/
   creating: archive2/BBC News Summary/BBC News Summary/News Articles/business/
  inflating: archive2/BBC News Summary/BBC News Summary/News Articles/business/001.txt  
  inflating: archive2/BBC News Summary/BBC News Summary/News Articles/business/002.txt  
  inflating: archive2/BBC News Summary/BBC News Summary/News Articles/business/003.txt  
  inflating: archive2/BBC News Summary/BBC News Summary/News Articles/business/004.txt  
  inflating: archive2/BBC News Summary/BBC News Summary/News Articles/business/005.txt  
  inflating: archive2/BBC News Summary/BBC News Summary/News Articles/business/006.txt  
  inflating: archive2/BBC News Summary/BBC News Summary/News Articles/business/007.txt  
  inflating: archive2/BBC News Summary/BBC News Summary/News Articles/business/008.txt  
  inflating: arch

In [1]:
!pip install spacy
!python -m spacy download en_core_web_lg # Spacy small model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-24 10:07:33.662474: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 10:07:34.523964: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-24 10:07:34.524076: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could no

In [2]:
# Imports
import os
from os.path import join
import numpy as np
import pandas as pd
import spacy 
nlp = spacy.load("en_core_web_lg")

#NLTK libraries and others
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
nltk.download('omw-1.4')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [111]:
import copy
f = open( '/content/archive2/BBC News Summary/BBC News Summary/News Articles/tech/002.txt' , "r")
text = f.read()
sents = nltk.sent_tokenize(text)
title = sents[0].split('\n\n',1)[0]
sents[0] = sents[0].split('\n\n',1)[1]
len_doc = len(sents)
print("Length doc(sentences): ", len_doc)

# Get reference summary
r = open( '/content/archive2/BBC News Summary/BBC News Summary/Summaries/tech/002.txt' , "r")
reference = r.read()
doc = nlp(reference)
sentence_tokens = [[token.text for token in sent] for sent in doc.sents]
n = len(sentence_tokens) #number of sentences in the reference summary 
print("length of summary: ",n)

Length doc(sentences):  19
length of summary:  4


In [112]:
# Stage to remove stopwords
stopwords = nlp.Defaults.stop_words
processed_text = []
words = []
for line in sents:
  lst=[]
  words = line.split(" ")
  for w in words:
    if w.lower() not in stopwords: # Checking whether the word is not present in the stopword list.
      lst.append(w)
  line =' '.join(lst) #Text without stopwords
  processed_text.append(line) 

print(sents[9])
print(processed_text[9])

China also tries to dictate the types of computer games people can play to limit the amount of violence people are exposed to.
China tries dictate types computer games people play limit violence people exposed to.


In [113]:
#Sentence pre-processing
# remove punctuations, numbers and special characters
for i in processed_text:
  i = pd.Series(i[1]).str.replace("[^a-zA-Z]", " ", regex=True)

#make alphabets lowercase
for i in processed_text:
  i = [s.lower() for s in i]
  i = ' '.join(i)

print(processed_text[9])

China tries dictate types computer games people play limit violence people exposed to.


## TextRank with word vectors

In [7]:
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2023-02-24 10:03:01--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-02-24 10:03:01--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-02-24 10:03:01--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [8]:
! unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [114]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [115]:
# Calculate value vectors for all sentences
words = []
sentence_vectors = []
for line in processed_text:
  words = line.split(" ")
  sum = np.zeros(100)
  for w in words:
    v = word_embeddings.get(w,np.zeros(100,))/(len(words)+0.001)
    sum = np.add(sum,v)
  sentence_vectors.append(sum)

In [116]:
# similarity matrix
sim_mat = np.zeros([len(processed_text), len(processed_text)])

from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(processed_text)):
  for j in range(len(processed_text)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [12]:
! pip install networkx[default]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 22.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


In [117]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

max_ind = max(scores, key=scores.get)
max_score = scores[max(scores, key=scores.get)]
#Normalize scores where max score has the value 1
for i in range(len(scores)):
  scores[i] = scores[i]/max_score

ranked_scores = sorted(((scores[i],s) for i,s in enumerate(scores)), reverse=True)

# Extract top n sentences as the summary
summary_ids = [] #Best approach: select the ranked sentences
for i in ranked_scores[:n]: 
  summary_ids.append(i[1])
summary_ids = sorted(summary_ids, reverse=False)

summary = []
for i in summary_ids:
  summary.append(sents[i])

#Convert list of sentences, to plain text (Used for ROUGE evaluation)
#Extract also ids of selected sentences (for tsv file)
summary_text =''.join(summary)

print("TOTAL SENTENCES OF ORIGINAL TEXT")
print(len(scores))
print("TOTAL SENTENCES EXTRACTED")
print(len(summary))

sum_spacy = nlp(summary_text)
word_len = max([c for c,i in enumerate(sum_spacy)])
print("LENGTH (QUANTITY OF WORDS)")
print(word_len)

TOTAL SENTENCES OF ORIGINAL TEXT
19
TOTAL SENTENCES EXTRACTED
4
LENGTH (QUANTITY OF WORDS)
90


In [ ]:
ranked_scores

In [120]:
summary_text

'Chinese net cafes operate under a set of strict guidelines and many of those most recently closed broke rules that limit how close they can be to schools.This is not the first time that the Chinese government has moved against net cafes that are not operating within its strict guidelines.During the crackdown following the blaze authorities moved to clean up net cafes and demanded that all of them get permits to operate.At the same time it introduced new controls to block overseas sex sites.'

In [119]:
reference

'In 2004, the Chinese government shut down 12,575 net cafes in the last three months of the year due to illegal operation and violating regulations, such as being too close to schools. The government is concerned about the influence of net cafes on young people and has introduced rules to limit their use. All net cafes are required to use software that controls website access and logs user activity. The Chinese government has a history of cracking down on net cafes and internet use it deems immoral, including shutting down websites and blocking overseas sex sites.\n'

In [121]:
temp = nlp(reference)
c=0
for c,i in enumerate(temp):
  c+=1
print("TOTAL words of reference")
print(c)

d=0
for c,i in enumerate(temp.sents):
  d+=1
print("TOTAL sentences of reference")
print(d)


TOTAL words of reference
104
TOTAL sentences of reference
4


##Evaluation with ROUGE

In [12]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [122]:
from rouge import Rouge
ROUGE = Rouge()
ROUGE.get_scores(summary_text, reference)

[{'rouge-1': {'r': 0.3333333333333333,
   'p': 0.36507936507936506,
   'f': 0.3484848434951791},
  'rouge-2': {'r': 0.08695652173913043,
   'p': 0.0963855421686747,
   'f': 0.0914285664417962},
  'rouge-l': {'r': 0.30434782608695654,
   'p': 0.3333333333333333,
   'f': 0.3181818131921489}}]

In [ ]:
#TODO